In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget
import pprint
import seaborn as sns 
sns.set()
#sns.reset_orig


## Part 1: Getting the Data
##




In [2]:
#######################
import myauz
from myauz.myalpha_funcs import (
    read_data,
    persist_data,
    update_csv,
    compose_portfolio,
    retrieveDF,
    string2date,
    retrievePF,
    initialize_df,
    time_sleep,
    create_path_list,
    refresh_db,
    get_daily_symbol,
)

#import importlib
#importlib.reload(myauz.myalpha_funcs)

from myauz.myalpha_libs_universal import StocksDb

import pprint


from datetime import date
import datetime
import pandas as pd



In [3]:
alphaDB = StocksDb()
alphaDB.check_path()
print("\n", alphaDB.api_key_alpha)


api_key_alpha = alphaDB.api_key_alpha
root_path = alphaDB.path
print('root_path: ',root_path)

read_from_alphavantage = False
rename_column = True


usecols = ["timestamp", "adjusted_close"]
startd = "2010-01-01"
endd = "2020-09-17"

print('usecols:', usecols)
print('startd',startd)
print('endd',endd)


Directory  ../data  already exists

 77K8XPSR3XWWZJ80
root_path:  ..
usecols: ['timestamp', 'adjusted_close']
startd 2010-01-01
endd 2020-09-17


In [4]:
symbol_list = [
   "MSFT",
   "GOOG",    
   "DHR",
   "EMR",
   "ATR",
   "HXGBY",
   "NVS",
   "NVZMY",    
   "CL",
   "MCD",    
   "HEINY",
   "LDSVF",  
]

path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)

bool_list_all = [
   True, #"MSFT", microsoft    
   True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   True, #"DHR", danaher
   True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
   True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
   True, #"HEINY", heineken
   True, #"LDSVF", lindt & spruengli    
]

bool_list_high_tech = [
   True, #"MSFT", microsoft    
   True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   False, #"DHR", danaher
   False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
   False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
   False, #"HEINY", heineken
   False, #"LDSVF", lindt & spruengli    
]

bool_list_industry = [
   False, #"MSFT", microsoft    
   False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   True, #"DHR", danaher
   True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
   False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
   False, #"HEINY", heineken
   False, #"LDSVF", lindt & spruengli    
]

bool_list_consumer = [
   False, #"MSFT", microsoft    
   False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   False, #"DHR", danaher
   False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
   True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
   True, #"HEINY", heineken
   True, #"LDSVF", lindt & spruengli    
]

colors = [
    'magenta',
    'lime',
#   'tab:orange',
    'blue',
    'black',
    'slategrey',
    'tab:brown',
    'tab:olive',
    'darkviolet',
    'red',
    'turquoise',
    'forestgreen',
    'gold',
]

color_dict = dict(zip(symbol_list, colors))
print('\n')
print(color_dict)


{'ATR': '../data/ATR/daily_ATR.csv',
 'CL': '../data/CL/daily_CL.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'EMR': '../data/EMR/daily_EMR.csv',
 'GOOG': '../data/GOOG/daily_GOOG.csv',
 'HEINY': '../data/HEINY/daily_HEINY.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'LDSVF': '../data/LDSVF/daily_LDSVF.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}


{'MSFT': 'magenta', 'GOOG': 'lime', 'DHR': 'blue', 'EMR': 'black', 'ATR': 'slategrey', 'HXGBY': 'tab:brown', 'NVS': 'tab:olive', 'NVZMY': 'darkviolet', 'CL': 'red', 'MCD': 'turquoise', 'HEINY': 'forestgreen', 'LDSVF': 'gold'}


In [5]:
# filter columns(symbols) in dataframe  by bool_list(symbols = True - this is the _included list)
def pf_filtered(_pf, _startd, _included):
    if not (_startd):
        _startd = '2020-01-01'
    if _included is []:
        return _pf
    else: 
        return _pf.loc[startd:,_included]

from itertools import compress
def symbol_list_filtered(_symbol_list,_included):
    #filter the symbol_list
    #symbol_list_filtered = [i for indx,i in enumerate(symbol_list) if included[indx]]
    return list(compress(_symbol_list, _included))

def determine_earliest_notation(_pf):
    """
    input:
    ------
         portfolio - columns = symbol  - import also symbol_list
    
    output:
    -------
         dataframe with one row containing earliest notation in time of symbol in portfolio
    
    """
    _pf_tmp = _pf.copy()
    _pf_tmp.fillna(0, inplace=True)
    _symbol_list = list(_pf_tmp.columns)
    _df_aux_0 = _pf_tmp.iloc[0]
    
    for symbol in _symbol_list:
        _df_aux_0[symbol] = (_pf_tmp[_pf_tmp[symbol] > 0].iloc[0]).loc[symbol]  
    return _df_aux_0

In [6]:
print('root_path:',root_path)

root_path: ..


In [7]:
### create dataframe with portfolio adjusted close
usecols = ["timestamp", "adjusted_close"]
path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)
pf = retrievePF(symbol_list, path_list, startd, endd, usecols, rename_column)
print(pf.index.name)
pf.index.rename('DATE', inplace=True)
print(pf.index.name)
pf.tail()

{'ATR': '../data/ATR/daily_ATR.csv',
 'CL': '../data/CL/daily_CL.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'EMR': '../data/EMR/daily_EMR.csv',
 'GOOG': '../data/GOOG/daily_GOOG.csv',
 'HEINY': '../data/HEINY/daily_HEINY.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'LDSVF': '../data/LDSVF/daily_LDSVF.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}
None
DATE


,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2020-09-10,205.37,1532.02,203.09,67.51,117.19,73.4300,88.52,63.730,76.41,217.24,45.445,8559.35
2020-09-11,204.03,1520.72,204.87,67.92,118.36,74.6600,89.88,65.590,76.75,218.00,45.420,8559.35
2020-09-14,205.41,1519.28,207.93,68.19,119.08,75.4694,90.03,65.275,77.15,220.56,45.990,8588.50
2020-09-15,208.78,1541.44,208.69,68.41,118.33,75.5200,91.00,65.220,76.94,222.37,46.030,8450.00
2020-09-16,205.05,1520.90,207.05,68.68,117.77,76.2800,89.85,65.980,76.91,224.81,45.650,8450.00


In [8]:
df_returns = (pf/pf.shift(1))-1
#ret = pf.pct_change(1)
pprint.pprint(df_returns.head())

print('___________________')

df1_returns = pf_filtered(df_returns, startd, bool_list_high_tech)


print('\n high_tech:')
#df1_returns = (pf1/pf1.shift(1))-1
pprint.pprint(df1_returns.head(3))


print('\n industry:')
#df2_returns = (pf2/pf2.shift(1))-1
df2_returns = pf_filtered(df_returns, startd, bool_list_industry)
pprint.pprint(df2_returns.head(3))


print('\n consumer: ')
#df3_returns = (pf3/pf3.shift(1))-1
df3_returns = pf_filtered(df_returns, startd, bool_list_consumer)
pprint.pprint(df3_returns.head(3))

                MSFT  GOOG       DHR       EMR       ATR  HXGBY       NVS  \
DATE                                                                        
2010-01-04       NaN   NaN       NaN       NaN       NaN    NaN       NaN   
2010-01-05  0.000323   NaN -0.001198  0.001614 -0.001104    NaN -0.000951   
2010-01-06 -0.006138   NaN  0.001866  0.008984  0.015194    NaN -0.003613   
2010-01-07 -0.010337   NaN  0.008245  0.000000  0.000000    NaN -0.008784   
2010-01-08  0.006833   NaN  0.013977  0.005706  0.000815    NaN  0.008089   

               NVZMY        CL       MCD  HEINY     LDSVF  
DATE                                                       
2010-01-04       NaN       NaN       NaN    NaN       NaN  
2010-01-05  0.004066  0.008089 -0.007645    NaN -0.008482  
2010-01-06 -0.026532 -0.002275 -0.013643    NaN  0.008555  
2010-01-07  0.011547 -0.003961  0.007321    NaN  0.000000  
2010-01-08 -0.017806 -0.017953 -0.000969    NaN -0.038393  
___________________

 high_tech:
       

In [9]:
df_cum_returns = initialize_df(symbol_list, startd, endd)
#print(df_cum_returns.head())
#print(df_returns)

df_cum_returns = (1+df_returns).cumprod()
###############achtung dataframe wird auf naechtes datum ohne NaN gesetzt
#df_cum_returns.dropna(inplace=True)
#################
df_cum_returns.head()
#df_cum_returns.head()

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,1.000323,NaN,0.998802,1.001614,0.998896,NaN,0.999049,1.004066,1.008089,0.992355,NaN,0.991518
2010-01-06,0.994183,NaN,1.000666,1.010613,1.014073,NaN,0.995440,0.977426,1.005795,0.978816,NaN,1.000000
2010-01-07,0.983906,NaN,1.008917,1.010613,1.014073,NaN,0.986696,0.988713,1.001811,0.985982,NaN,1.000000
2010-01-08,0.990629,NaN,1.023019,1.016380,1.014900,NaN,0.994677,0.971109,0.983826,0.985027,NaN,0.961607


In [10]:
invested_dict = {
    "MSFT": float(1297.59),
    "GOOG": float(2111),
    "DHR": float(3168.48),
    "EMR": float(418.10),
    "ATR": float(289.49),
    "HXGBY": float(1060.39),
    "NVS": float(404.42),
    "NVZMY": float(1049.95),
    "CL": float(390.03),
    "MCD": float(872.35),  
    "HEINY": float(575.60),     
    "LDSVF": float(6942.25),   
}


def calc_weights(_invested_dict):
    
    _invested_list = list(_invested_dict.values())
    _invested_array =np.array(_invested_list)
    _total_investment=np.sum(_invested_array)
    #print("total investment: ")
    #pprint.pprint(_total_investment)
    _weights_dict = {}
    for symbol, investment in _invested_dict.items():
        _weights_dict[symbol] = investment / _total_investment
        
    return _total_investment, _weights_dict

total_investment, weights_dict = calc_weights(invested_dict)

print("total investment:",total_investment)

weight_sum = 0
for symbol, weight in weights_dict.items():
    pprint.pprint((symbol, weight))
    weight_sum += weight

print(weight_sum)

"""
weights = [
   .1,  #"MSFT",
   .1,  #"GOOG",    
   .1, #"DHR",
   .1,  #"EMR",
   .1,  #"ATR",
   .1,  #"HXGBY",
   .1,  #"NVS",
   .1,  #"NVZMY",    
   .1,  #"CL",
   .05,  #"MCD",    
   .03,  #"HEINY",
   .02,  #"LDSVF",  
]
"""



total investment: 18579.65
('MSFT', 0.0698393134423953)
('GOOG', 0.1136189325417863)
('DHR', 0.17053496702036905)
('EMR', 0.022503114967181836)
('ATR', 0.01558102547679854)
('HXGBY', 0.057072657450490186)
('NVS', 0.021766825532235537)
('NVZMY', 0.05651075235539959)
('CL', 0.020992322245036905)
('MCD', 0.04695190705960554)
('HEINY', 0.030980131487945144)
('LDSVF', 0.373648050420756)
0.9999999999999998


'\nweights = [\n   .1,  #"MSFT",\n   .1,  #"GOOG",    \n   .1, #"DHR",\n   .1,  #"EMR",\n   .1,  #"ATR",\n   .1,  #"HXGBY",\n   .1,  #"NVS",\n   .1,  #"NVZMY",    \n   .1,  #"CL",\n   .05,  #"MCD",    \n   .03,  #"HEINY",\n   .02,  #"LDSVF",  \n]\n'

In [11]:
#ensure that weights vector is in correct order as symbol_list
weights = np.array(list(weights_dict.values()))
print(weights)

df_pos_value = df_cum_returns * weights * total_investment
df_pos_value.ffill(inplace=True)
df_pos_value['TOTAL_POS'] = df_pos_value.sum(axis=1)

print('\n _________________________________________________________________________________')

df_pos_value = df_pos_value[1:]
df_pos_value['TOTAL_POS']

[0.06983931 0.11361893 0.17053497 0.02250311 0.01558103 0.05707266
 0.02176683 0.05651075 0.02099232 0.04695191 0.03098013 0.37364805]

 _________________________________________________________________________________


DATE
2010-01-05    14771.124327
2010-01-06    14793.969815
2010-01-07    14819.788651
2010-01-08    14586.206675
2010-01-11    14641.559481
                  ...     
2020-09-10    83533.538408
2020-09-11    83839.111493
2020-09-14    84456.922776
2020-09-15    84412.373310
2020-09-16    84050.555474
Name: TOTAL_POS, Length: 2694, dtype: float64

In [14]:
plt.figure()
plt.title("total position of portfolio")
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')





df_pos_value['TOTAL_POS'].plot(color=['royalblue'], figsize=(16,10), linewidth=1.25);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
color_list = [color_dict.get(x, _) for x in pf.columns]

plt.title('all stocks in portfolio')
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid( which='minor', linestyle=':', linewidth='0.5', color='gray')


df_pos_value.plot(color=color_list,figsize=(16,10), linewidth=1.5);
